# Basic NMT with Tensorflow seq2seq

## Making the imports

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
import math
import numpy as np
import os
import random
import tensorflow as tf
from matplotlib import pylab
from collections import Counter
import csv

# Seq2Seq Items
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.ops.rnn_cell import LSTMCell
from tensorflow.python.ops.rnn_cell import MultiRNNCell
from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.python.layers.core import Dense

## Defining some hyperparameters of the model

In [4]:
vocab_size= 50000
num_units = 128
input_size = 128
batch_size = 16
source_sequence_length=40
target_sequence_length=60
decoder_type = 'basic' # could be basic or attention
sentences_to_read = 50000

## Loading vocabularies

In [5]:
src_dictionary = dict()
with open('vocab.50K.de.txt', encoding='utf-8') as f:
    for line in f:
        #we are discarding last char as it is new line char
        src_dictionary[line[:-1]] = len(src_dictionary)

src_reverse_dictionary = dict(zip(src_dictionary.values(),src_dictionary.keys()))

print('Source')
print('\t',list(src_dictionary.items())[:10])
print('\t',list(src_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(src_dictionary))

tgt_dictionary = dict()
with open('vocab.50K.en.txt', encoding='utf-8') as f:
    for line in f:
        #we are discarding last char as it is new line char
        tgt_dictionary[line[:-1]] = len(tgt_dictionary)

tgt_reverse_dictionary = dict(zip(tgt_dictionary.values(),tgt_dictionary.keys()))

print('Target')
print('\t',list(tgt_dictionary.items())[:10])
print('\t',list(tgt_reverse_dictionary.items())[:10])
print('\t','Vocabulary size: ', len(tgt_dictionary))


Source
	 [('ausgestattetes', 16818), ('Firmengeschichte', 40229), ('fürchterlichen', 39481), ('Melancholie', 49659), ('Route', 5970), ('Lobbyisten', 11946), ('zuweilen', 9260), ('Computer-', 32843), ('Produktreihe', 27955), ('Nervensystem', 32983)]
	 [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, ','), (4, '.'), (5, 'die'), (6, 'der'), (7, 'und'), (8, 'in'), (9, 'zu')]
	 Vocabulary size:  50000
Target
	 [('vigour', 16936), ('Markets', 11144), ('screw', 10318), ('Route', 9932), ('Sates', 45188), ('roar', 37596), ('biographical', 29851), ('proportioned', 35062), ('Gozo', 26604), ('base', 1257)]
	 [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, 'the'), (4, ','), (5, '.'), (6, 'of'), (7, 'and'), (8, 'to'), (9, 'in')]
	 Vocabulary size:  50000


## Loading Sentences (English and German)

In [6]:
source_sent = []
target_sent = []

test_source_sent = []
test_target_sent = []


with open('train.de', encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        # discarding first 20 translations as there was some
        # english to english translations found in the first few. which are wrong
        if l_i<50:
            continue
        source_sent.append(line)
        if len(source_sent)>=sentences_to_read:
            break
        
            
with open('train.en', encoding='utf-8') as f:
    for l_i, line in enumerate(f):
        if l_i<50:
            continue
        
        target_sent.append(line)
        if len(target_sent)>=sentences_to_read:
            break
        
            
assert len(source_sent)==len(target_sent),'Source: %d, Target: %d'%(len(source_sent),len(target_sent))

print('Sample translations (%d)'%len(source_sent))
for i in range(0,sentences_to_read,10000):
    print('(',i,') DE: ', source_sent[i])
    print('(',i,') EN: ', target_sent[i])


Sample translations (50000)
( 0 ) DE:  Heute verstehen sich QuarkXPress ® 8 , Photoshop ® und Illustrator ® besser als jemals zuvor . Dank HTML und CSS ­ können Anwender von QuarkXPress inzwischen alle Medien bedienen , und das unabhängig von Anwendungen der Adobe ® Creative Suite ® wie Adobe Flash ® ( SWF ) und Adobe Dreamweaver ® .

( 0 ) EN:  Today , QuarkXPress ® 8 has tighter integration with Photoshop ® and Illustrator ® than ever before , and through standards like HTML and CSS , QuarkXPress users can publish across media both independently and alongside Adobe ® Creative Suite ® applications like Adobe Flash ® ( SWF ) and Adobe Dreamweaver ® .

( 10000 ) DE:  Es existieren Busverbindungen in nahezu jeden Ort der Provence ( eventuell mit Umsteigen in Aix ##AT##-##AT## en ##AT##-##AT## Provence ) , allerdings sollte beachtet werden , dass die letzten Busse abends ca. um 19 Uhr fahren .

( 10000 ) EN:  As always in France those highways are expensive but practical , comfortable and

## Let's analyse some statistics 

In [7]:
def split_to_tokens(sent,is_source):
    #sent = sent.replace('-',' ')
    sent = sent.replace(',',' ,')
    sent = sent.replace('.',' .')
    sent = sent.replace('\n',' ') 
    
    sent_toks = sent.split(' ')
    for t_i, tok in enumerate(sent_toks):
        if is_source:
            if tok not in src_dictionary.keys():
                sent_toks[t_i] = '<unk>'
        else:
            if tok not in tgt_dictionary.keys():
                sent_toks[t_i] = '<unk>'
    return sent_toks

# Let us first look at some statistics of the sentences
source_len = []
source_mean, source_std = 0,0
for sent in source_sent:
    source_len.append(len(split_to_tokens(sent,True)))

print('(Source) Sentence mean length: ', np.mean(source_len))
print('(Source) Sentence stddev length: ', np.std(source_len))

target_len = []
target_mean, target_std = 0,0
for sent in target_sent:
    target_len.append(len(split_to_tokens(sent,False)))

print('(Target) Sentence mean length: ', np.mean(target_len))
print('(Target) Sentence stddev length: ', np.std(target_len))

(Source) Sentence mean length:  26.35934
(Source) Sentence stddev length:  13.9681614669
(Target) Sentence mean length:  28.58758
(Target) Sentence stddev length:  15.1544201388


## Add the special tokens and make all sentences same length (for batch-processing)

In [8]:
train_inputs = []
train_outputs = []
train_inp_lengths = []
train_out_lengths = []

max_tgt_sent_lengths = 0

src_max_sent_length = 41
tgt_max_sent_length = 61
for s_i, (src_sent, tgt_sent) in enumerate(zip(source_sent,target_sent)):
    
    src_sent_tokens = split_to_tokens(src_sent,True)
    tgt_sent_tokens = split_to_tokens(tgt_sent,False)
        
    num_src_sent = []
    for tok in src_sent_tokens:
        num_src_sent.append(src_dictionary[tok])

    num_src_set = num_src_sent[::-1] # we reverse the source sentence. This improves performance
    num_src_sent.insert(0,src_dictionary['<s>'])
    train_inp_lengths.append(min(len(num_src_sent)+1,src_max_sent_length))
    
    # append until the sentence reaches max length
    if len(num_src_sent)<src_max_sent_length:
        num_src_sent.extend([src_dictionary['</s>'] for _ in range(src_max_sent_length - len(num_src_sent))])
    # if more than max length, truncate the sentence
    elif len(num_src_sent)>src_max_sent_length:
        num_src_sent = num_src_sent[:src_max_sent_length]
    assert len(num_src_sent)==src_max_sent_length,len(num_src_sent)

    train_inputs.append(num_src_sent)

    num_tgt_sent = [tgt_dictionary['</s>']]
    for tok in tgt_sent_tokens:
        num_tgt_sent.append(tgt_dictionary[tok])
    
    train_out_lengths.append(min(len(num_tgt_sent)+1,tgt_max_sent_length))
    
    if len(num_tgt_sent)<tgt_max_sent_length:
        num_tgt_sent.extend([tgt_dictionary['</s>'] for _ in range(tgt_max_sent_length - len(num_tgt_sent))])
    elif len(num_tgt_sent)>tgt_max_sent_length:
        num_tgt_sent = num_tgt_sent[:tgt_max_sent_length]
    
    train_outputs.append(num_tgt_sent)
    assert len(train_outputs[s_i])==tgt_max_sent_length, 'Sent length needs to be 60, but is %d'%len(binned_outputs[s_i])    

assert len(train_inputs)  == len(source_sent),\
        'Size of total bin elements: %d, Total sentences: %d'\
                %(len(train_inputs),len(source_sent))

print('Max sent lengths: ', max_tgt_sent_lengths)


train_inputs = np.array(train_inputs, dtype=np.int32)
train_outputs = np.array(train_outputs, dtype=np.int32)
train_inp_lengths = np.array(train_inp_lengths, dtype=np.int32)
train_out_lengths = np.array(train_out_lengths, dtype=np.int32)
print('Samples from bin')
print('\t',[src_reverse_dictionary[w]  for w in train_inputs[0,:].tolist()])
print('\t',[tgt_reverse_dictionary[w]  for w in train_outputs[0,:].tolist()])
print('\t',[src_reverse_dictionary[w]  for w in train_inputs[10,:].tolist()])
print('\t',[tgt_reverse_dictionary[w]  for w in train_outputs[10,:].tolist()])
print()
print('\tSentences ',train_inputs.shape[0])

Max sent lengths:  0
Samples from bin
	 ['<s>', 'Heute', 'verstehen', 'sich', 'QuarkXPress', '®', '8', '<unk>', ',', 'Photoshop', '®', 'und', 'Illustrator', '®', 'besser', 'als', 'jemals', 'zuvor', '<unk>', '.', 'Dank', 'HTML', 'und', 'CSS', '\xad', 'können', 'Anwender', 'von', 'QuarkXPress', 'inzwischen', 'alle', 'Medien', 'bedienen', '<unk>', ',', 'und', 'das', 'unabhängig', 'von', 'Anwendungen', 'der']
	 ['</s>', 'Today', '<unk>', ',', 'QuarkXPress', '®', '8', 'has', 'tighter', 'integration', 'with', 'Photoshop', '®', 'and', 'Illustrator', '®', 'than', 'ever', 'before', '<unk>', ',', 'and', 'through', 'standards', 'like', 'HTML', 'and', 'CSS', '<unk>', ',', 'QuarkXPress', 'users', 'can', 'publish', 'across', 'media', 'both', 'independently', 'and', 'alongside', 'Adobe', '®', 'Creative', 'Suite', '®', 'applications', 'like', 'Adobe', 'Flash', '®', '(', 'SWF', ')', 'and', 'Adobe', 'Dreamweaver', '®', '<unk>', '.', '<unk>', '</s>']
	 ['<s>', 'Erstellen', 'Sie', 'einen', 'Rahmen', 'und'

## Batch Data Generator 

In [9]:
input_size = 128

class DataGeneratorMT(object):
    
    def __init__(self,batch_size,num_unroll,is_source):
        self._batch_size = batch_size
        self._num_unroll = num_unroll
        self._cursor = [0 for offset in range(self._batch_size)]
        
        
        self._src_word_embeddings = np.load('de-embeddings.npy')
        
        self._tgt_word_embeddings = np.load('en-embeddings.npy')
        
        self._sent_ids = None
        
        self._is_source = is_source
        
                
    def next_batch(self, sent_ids, first_set):
        
        if self._is_source:
            max_sent_length = src_max_sent_length
        else:
            max_sent_length = tgt_max_sent_length
        batch_labels_ind = []
        batch_data = np.zeros((self._batch_size),dtype=np.float32)
        batch_labels = np.zeros((self._batch_size),dtype=np.float32)
        
        for b in range(self._batch_size):
            
            sent_id = sent_ids[b]
            
            if self._is_source:
                sent_text = train_inputs[sent_id]
                             
                batch_data[b] = sent_text[self._cursor[b]]
                batch_labels[b]=sent_text[self._cursor[b]+1]

            else:
                sent_text = train_outputs[sent_id]
                
                # We cannot avoid having two different embedding vectors for <s> token
                # in soruce and target languages
                # Therefore, if the symbol appears, we always take the source embedding vector
                if sent_text[self._cursor[b]]!=src_dictionary['<s>']:
                    batch_data[b] = sent_text[self._cursor[b]]
                else:
                    batch_data[b] = sent_text[self._cursor[b]]
                batch_labels[b] = sent_text[self._cursor[b]+1]

            self._cursor[b] = (self._cursor[b]+1)%(max_sent_length-1)
                                    
        return batch_data,batch_labels
        
    def unroll_batches(self,sent_ids):
        
        if sent_ids is not None:
            
            self._sent_ids = sent_ids
            
            #if self._is_source:
                # we dont star at the very beginning, becaues the very beginning is a bunch of </s> symbols.
                # so we start from the middel s.t we get a minimum number of </s> symbols in our training data
                # this is only needed for source language
                #self._cursor = ((start_indices_for_bins[bin_id][self._sent_ids]//self._num_unroll)*self._num_unroll).tolist()
            #else:
            self._cursor = [0 for _ in range(self._batch_size)]
                
        unroll_data,unroll_labels = [],[]
        inp_lengths = None
        for ui in range(self._num_unroll):
            # The first batch in any batch of captions is different
            if self._is_source:
                data, labels = self.next_batch(self._sent_ids, False)
            else:
                data, labels = self.next_batch(self._sent_ids, False)
                    
            unroll_data.append(data)
            unroll_labels.append(labels)
            inp_lengths = train_inp_lengths[sent_ids]
        return unroll_data, unroll_labels, self._sent_ids, inp_lengths
    
    def reset_indices(self):
        self._cursor = [0 for offset in range(self._batch_size)]
        
# Running a tiny set to see if the implementation correct
dg = DataGeneratorMT(batch_size=5,num_unroll=40,is_source=True)
u_data, u_labels, _, _ = dg.unroll_batches([0,1,2,3,4])

print('Source data')
for _, lbl in zip(u_data,u_labels):
    print([src_reverse_dictionary[w] for w in lbl.tolist()])

        
# Running a tiny set to see if the implementation correct
dg = DataGeneratorMT(batch_size=5,num_unroll=60,is_source=False)
u_data, u_labels, _, _ = dg.unroll_batches([0,2,3,4,5])
print('\nTarget data batch (first time)')
for d_i,(_, lbl) in enumerate(zip(u_data,u_labels)):
    #if d_i>5 and d_i < 35:
    #    continue

    print([tgt_reverse_dictionary[w] for w in lbl.tolist()])

print('\nTarget data batch (non-first time)')
u_data, u_labels, _, _ = dg.unroll_batches(None)
for d_i,(_, lbl) in enumerate(zip(u_data,u_labels)):
    
    #if d_i>5 and d_i < 35:
    #    continue
        
    print([tgt_reverse_dictionary[w] for w in lbl.tolist()])


Source data
['Heute', 'Hier', 'Sie', 'Häufig', 'In']
['verstehen', 'erfahren', 'werden', 'wird', 'diesem']
['sich', 'Sie', 'überrascht', 'die', 'Abschnitt']
['QuarkXPress', '<unk>', 'sein', 'Meinung', 'erläutern']
['®', ',', '<unk>', 'vertreten', 'wir']
['8', 'wie', ',', '<unk>', '<unk>']
['<unk>', 'Sie', 'wie', ',', ',']
[',', 'Creative', 'einfach', 'dass', 'wann']
['Photoshop', 'Suite', 'sich', 'QuarkXPress', 'Sie']
['®', '2', 'mit', '8', 'für']
['und', 'und', 'Quark', 'von', 'Ihre']
['Illustrator', 'Creative', 'das', 'allen', 'Bilder']
['®', 'Suite', 'volle', 'heute', 'das']
['besser', '3', 'Potenzial', 'verfügbaren', 'PSD']
['als', 'am', 'Ihrer', 'Layout', '##AT##-##AT##']
['jemals', 'besten', 'Design', '##AT##-##AT##', 'Format']
['zuvor', 'zusammen', '##AT##-##AT##', 'Programmen', 'verwenden']
['<unk>', 'mit', 'Software', 'die', 'sollten']
['.', 'QuarkXPress', 'erschließen', 'beste', 'und']
['Dank', 'nutzen', 'lässt', 'Integration', 'wie']
['HTML', 'können', '<unk>', 'mit', 'Sie']

## Inputs Outputs Masks

In [10]:
tf.reset_default_graph()

enc_train_inputs = []
dec_train_inputs = []

# Need to use pre-trained word embeddings
encoder_emb_layer = tf.convert_to_tensor(np.load('de-embeddings.npy'))
decoder_emb_layer = tf.convert_to_tensor(np.load('en-embeddings.npy'))

# Defining unrolled training inputs
for ui in range(source_sequence_length):
    enc_train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size],name='enc_train_inputs_%d'%ui))

dec_train_labels=[]
dec_label_masks = []
for ui in range(target_sequence_length):
    dec_train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size],name='dec_train_inputs_%d'%ui))
    dec_train_labels.append(tf.placeholder(tf.int32, shape=[batch_size],name='dec-train_outputs_%d'%ui))
    dec_label_masks.append(tf.placeholder(tf.float32, shape=[batch_size],name='dec-label_masks_%d'%ui))
    
encoder_emb_inp = [tf.nn.embedding_lookup(encoder_emb_layer, src) for src in enc_train_inputs]
encoder_emb_inp = tf.stack(encoder_emb_inp)

decoder_emb_inp = [tf.nn.embedding_lookup(decoder_emb_layer, src) for src in dec_train_inputs]
decoder_emb_inp = tf.stack(decoder_emb_inp)

enc_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_input_lengths')
dec_train_inp_lengths = tf.placeholder(tf.int32, shape=[batch_size],name='train_output_lengths')

## Encoder

In [11]:
encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)

encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_emb_inp, initial_state=initial_state,
    sequence_length=enc_train_inp_lengths, 
    time_major=True, swap_memory=True)

## Decoder

In [12]:
# Build RNN cell
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

projection_layer = Dense(units=vocab_size, use_bias=True)

# Helper
helper = tf.contrib.seq2seq.TrainingHelper(
    decoder_emb_inp, [tgt_max_sent_length-1 for _ in range(batch_size)], time_major=True)

# Decoder
if decoder_type == 'basic':
    decoder = tf.contrib.seq2seq.BasicDecoder(
        decoder_cell, helper, encoder_state,
        output_layer=projection_layer)
    
elif decoder_type == 'attention':
    decoder = tf.contrib.seq2seq.BahdanauAttention(
        decoder_cell, helper, encoder_state,
        output_layer=projection_layer)
    
# Dynamic decoding
outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
    decoder, output_time_major=True,
    swap_memory=True
)



## Loss and Predictions

In [13]:
logits = outputs.rnn_output

crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=dec_train_labels, logits=logits)
loss = (tf.reduce_sum(crossent*tf.stack(dec_label_masks)) / (batch_size*target_sequence_length))

train_prediction = outputs.sample_id

## Defining Optimizer with Gradient Clipping

In [15]:
print('Defining Optimizer')
# Adam Optimizer. And gradient clipping.
global_step = tf.Variable(0, trainable=False)
inc_gstep = tf.assign(global_step,global_step + 1)
learning_rate = tf.train.exponential_decay(
    0.01, global_step, decay_steps=10, decay_rate=0.9, staircase=True)

with tf.variable_scope('Adam'):
    adam_optimizer = tf.train.AdamOptimizer(learning_rate)

adam_gradients, v = zip(*adam_optimizer.compute_gradients(loss))
adam_gradients, _ = tf.clip_by_global_norm(adam_gradients, 25.0)
adam_optimize = adam_optimizer.apply_gradients(zip(adam_gradients, v))

with tf.variable_scope('SGD'):
    sgd_optimizer = tf.train.GradientDescentOptimizer(learning_rate)

sgd_gradients, v = zip(*sgd_optimizer.compute_gradients(loss))
sgd_gradients, _ = tf.clip_by_global_norm(sgd_gradients, 25.0)
sgd_optimize = sgd_optimizer.apply_gradients(zip(sgd_gradients, v))

sess = tf.InteractiveSession()

Defining Optimizer


## Running the NMT

In [16]:


if not os.path.exists('logs'):
    os.mkdir('logs')
log_dir = 'logs'

bleu_scores_over_time = []
loss_over_time = []
tf.global_variables_initializer().run()

src_word_embeddings = np.load('de-embeddings.npy')
tgt_word_embeddings = np.load('en-embeddings.npy')

# Defining data generators
enc_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=source_sequence_length,is_source=True)
dec_data_generator = DataGeneratorMT(batch_size=batch_size,num_unroll=target_sequence_length,is_source=False)

num_steps = 10001
avg_loss = 0

bleu_labels, bleu_preds = [],[]

print('Started Training')

for step in range(num_steps):

    # input_sizes for each bin: [40]
    # output_sizes for each bin: [60]
    print('.',end='')
    if (step+1)%100==0:
        print('')
        
    sent_ids = np.random.randint(low=0,high=train_inputs.shape[0],size=(batch_size))
    # ====================== ENCODER DATA COLLECTION ================================================
    
    eu_data, eu_labels, _, eu_lengths = enc_data_generator.unroll_batches(sent_ids=sent_ids)
    
    feed_dict = {}
    feed_dict[enc_train_inp_lengths] = eu_lengths
    for ui,(dat,lbl) in enumerate(zip(eu_data,eu_labels)):            
        feed_dict[enc_train_inputs[ui]] = dat                
    
    # ====================== DECODER DATA COLLECITON ===========================
    # First step we change the ids in a batch
    du_data, du_labels, _, du_lengths = dec_data_generator.unroll_batches(sent_ids=sent_ids)
    
    feed_dict[dec_train_inp_lengths] = du_lengths
    for ui,(dat,lbl) in enumerate(zip(du_data,du_labels)):            
        feed_dict[dec_train_inputs[ui]] = dat
        feed_dict[dec_train_labels[ui]] = lbl
        feed_dict[dec_label_masks[ui]] = (np.array([ui for _ in range(batch_size)])<du_lengths).astype(np.int32)
    
    # ======================= OPTIMIZATION ==========================
    if step < 10000:
        _,l,tr_pred = sess.run([adam_optimize,loss,train_prediction], feed_dict=feed_dict)
    else:
        _,l,tr_pred = sess.run([sgd_optimize,loss,train_prediction], feed_dict=feed_dict)
    tr_pred = tr_pred.flatten()
        
            
    if (step+1)%250==0:  
        
        print('Step ',step+1)

        print_str = 'Actual: '
        for w in np.concatenate(du_labels,axis=0)[::batch_size].tolist():
            print_str += tgt_reverse_dictionary[w] + ' '                    
            if tgt_reverse_dictionary[w] == '</s>':
                break
                      
        print(print_str)
        print()
        
        print_str = 'Predicted: '
        for w in tr_pred[::batch_size].tolist():
            print_str += tgt_reverse_dictionary[w] + ' '
            if tgt_reverse_dictionary[w] == '</s>':
                break
        print(print_str)
       
        print('\n')  
        
        rand_idx = np.random.randint(low=1,high=batch_size)
        print_str = 'Actual: '
        for w in np.concatenate(du_labels,axis=0)[rand_idx::batch_size].tolist():
            print_str += tgt_reverse_dictionary[w] + ' '
            if tgt_reverse_dictionary[w] == '</s>':
                break
        print(print_str)

            
        print()
        print_str = 'Predicted: '
        for w in tr_pred[rand_idx::batch_size].tolist():
            print_str += tgt_reverse_dictionary[w] + ' '
            if tgt_reverse_dictionary[w] == '</s>':
                break
        print(print_str)
        print()        
        
    avg_loss += l
    
    #sess.run(reset_train_state) # resetting hidden state for each batch
    
    if (step+1)%500==0:
        print('============= Step ', str(step+1), ' =============')
        print('\t Loss: ',avg_loss/500.0)
        
        loss_over_time.append(avg_loss/500.0)
             
        avg_loss = 0.0
        sess.run(inc_gstep)
            
        

Started Training
....................................................................................................
....................................................................................................
..................................................Step  250
Actual: We should consider all possible means of preventing those who still remain <unk> from getting the virus <unk> . But we must keep in mind too the 34 million Africans currently <unk> positive or ill with full ##AT##-##AT## blown Aids <unk> . <unk> </s> 

Predicted: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> </s> 


Actual: Greece and Greek Cyprus <unk> , for example <unk> , have as Eastern Orthodox a tradition as those parts of Eastern Europe that fell under communism <unk> , but because they remained in the West European orbit <unk> , these two countries have substantially modernise

Step  2500
Actual: The equipment <unk> / Printing production / Books <unk> , newspaper <unk> , <unk> <unk> . . . <unk> </s> 

Predicted: </s> 


Actual: There are a few pizzerias <unk> , mostly wood ##AT##-##AT## fired and quite acceptable <unk> . The <unk> chocolate sold at stores is delicious <unk> . <unk> </s> 

Predicted: </s> 

============= Step  2500  =============
	 Loss:  4.33138716078
....................................................................................................
....................................................................................................
..................................................Step  2750
Actual: Why not treat yourself to one of our luxurious Beauty treatments <unk> . Including <unk> <unk> , Make Up <unk> , Tan <unk> , Hands &amp; <unk> in our <unk> Suite <unk> . <unk> </s> 

Predicted: <unk> <unk> </s> 


Actual: The return value will vary based on the implementation <unk> . <unk> </s> 

Predicted: <unk> <unk> <unk> </s>

....................................................................................................
....................................................................................................
..................................................Step  5250
Actual: Video preview on our new fully ##AT##-##AT## automatic <unk> ##AT##-##AT## milling unit <unk> . <unk> </s> 

Predicted: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> </s> 


Actual: The guest reviews are submitted by our customers after their stay at <unk> <unk> . <unk> </s> 

Predicted: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> </s> 

..................................................
....................................................................................................
....................................................................................................
Step  5500
Actual: Description : Herbert Lodge Hostel is a large Georg

..................................................
....................................................................................................
....................................................................................................
Step  7500
Actual: Then I have a pair of Pentax Super Program for better image quality and I &apos;m currently experimenting with two Canon EOS 1 and <unk> <unk> <unk> . The best <unk> come with wide angles <unk> <unk> . . <unk> <unk> , and I also get stunning effects with 200 <unk> . . 300mm telephoto lenses on occasions <unk> . 

Predicted: <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> </s> 


Actual: According to the special report <unk> , security operators at the airport should check each bag before putting it into the plane ; however <unk> , some operators take advantage of the scanner machine to detect valuable objec

....................................................................................................
....................................................................................................
............................................

KeyboardInterrupt: 